# Cycle_GAN_Image2Image_Style_Transfer

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
drive_path = '/content/drive/MyDrive/COLAB_DRIVE/CycleGAN_Denoising_Code'

os.chdir(drive_path)
os.getcwd() 
!ls

Mounted at /content/drive
architecture	      data	__pycache__  Test.py	       utils.py
CycleGAN_Training.py  progress	summaries    training_weights


In [2]:
from architecture import generator, discriminator
# import tensorflow as tf
import utils
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

data_path = "data/Train"
n_epochs =  400#21#98#196
batch_size = 50


# read images
train_img, train_labels = utils.get_img_path(data_path)
num_train = int(len(train_img))
print('Number of Training images: ', num_train)

Instructions for updating:
non-resource variables are not supported in the long term
Number of Training images:  300


In [3]:
# train_vessel = train_labels
# train_imgs, train_vessels = get_train_batch(train_img, train_labels, num_train, batch_size, Training = True)
# input_images = image_files2arrs_normalize(train_img)
# train_imgs, train_vessels = utils.get_train_batch(train_img, train_labels, num_train, batch_size, Training = True)

In [4]:
# import PIL.Image as Image  # , ImageEnhance
# filenames = train_img
# target_size = (256, 256)
# # if len(img_shape) == 3:
# #     images_arr = np.zeros((len(filenames), img_shape[0], img_shape[1], img_shape[2]), dtype=np.float32)
# # elif len(img_shape) == 2:
# images_arr = np.zeros((len(filenames), 256, 256, 3), dtype=np.float32)

# for file_index in range(len(filenames)):
#   # print(file_index)
#   img = Image.open(filenames[file_index])
#   img = img.resize(target_size, Image.ANTIALIAS)
#   print(img)
#   # images_arr[file_index] = np.asarray(img).astype(np.float32)/255
    
    

#     ##################
#     # images_arr[file_index] = np.asarray(img).astype(np.float32)
#     # min_array = np.min(images_arr[file_index])
#     # max_array = np.max(images_arr[file_index])
#     # images_arr[file_index] = normalize(images_arr[file_index], min_array, max_array)
#     # THIS IS MY LINE
#     # images_arr[file_index]= image.extract_patches_2d(images_arr[file_index], patch_size=(256, 256), max_patches=1)
#     # images_arr[file_index] = images_arr[file_index] - 1


In [5]:
# images_arr.shape

In [6]:
# import PIL.Image as Image  # , ImageEnhance
# def image_files2arrs_normalize(filenames):
#     # img_shape = image_shape(filenames[0])
#     # images_arr = None
#     target_size = (256, 256, 3)
#     # if len(img_shape) == 3:
#     #     images_arr = np.zeros((len(filenames), img_shape[0], img_shape[1], img_shape[2]), dtype=np.float32)
#     # elif len(img_shape) == 2:
#     images_arr = np.zeros((len(filenames), 256, 256, 3), dtype=np.float32)

#     for file_index in range(len(filenames)):
#         img = Image.open(filenames[file_index])
#         img = img.resize(target_size, Image.ANTIALIAS)
#         images_arr[file_index] = np.asarray(img).astype(np.float32)/255
#         # images_arr[file_index] = np.asarray(img).astype(np.float32)
#         # min_array = np.min(images_arr[file_index])
#         # max_array = np.max(images_arr[file_index])
#         # images_arr[file_index] = normalize(images_arr[file_index], min_array, max_array)
#         # THIS IS MY LINE
#         # images_arr[file_index]= image.extract_patches_2d(images_arr[file_index], patch_size=(256, 256), max_patches=1)
#         # images_arr[file_index] = images_arr[file_index] - 1
#     return images_arr

In [7]:
# def get_train_batch(train_img, train_labels, num_train, batch_size, Training = True):
#     train_indices = np.random.choice(num_train, batch_size, replace=False)
#     if (Training):
#         train_indices2 = np.random.choice(num_train, batch_size, replace=False)

#     batch_size = len(train_indices)
#     batch_input_img_files, batch_target_img_files = [], []
#     if (Training):
#         for ind, idx in enumerate(train_indices):
#             batch_input_img_files.append(train_img[idx])
#             if (Training):
#                 batch_target_img_files.append(train_labels[train_indices2[ind]])
#             else:
#                 batch_target_img_files.append(train_labels[idx])
#     else:
#         for ind, idx in enumerate(train_indices):
#             batch_input_img_files.append(train_img[ind])
#             batch_target_img_files.append(train_labels[ind])
    
#     # print(batch_input_img_files)
#     # load images
#     input_images = image_files2arrs_normalize(batch_input_img_files)
#     target_images = image_files2arrs_normalize(batch_target_img_files)
#     # assert (np.min(vessel) == 0 and np.max(vessel) == 1)

#     all_input_images, all_target_images = [], []

#     for i in range(batch_size):
#         all_input_images.append(input_images[i])
#         all_target_images.append(target_images[i])


#     input_images = np.asarray(all_input_images)
#     target_images = np.asarray(all_target_images)

#     return input_images, target_images

In [8]:
# placeholders
a_real = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])  # unfixed-BF image (actual)
b_real = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])  # fixed-BF image (actual)

phase = tf.placeholder(tf.bool, name="phase")  # phase for training

fake_b = generator.generator(a_real, phase_in=phase, scope='a2b')  # generate fake-fixed-BF using real-unfixed BF
fake_a = generator.generator(b_real, phase_in=phase, scope='b2a')  # generate fake-unfixed-BF using real-fixed BF

fake_b_dis = discriminator.discriminator(fake_b, training=phase, scope='b')
fake_a_dis = discriminator.discriminator(fake_a, training=phase, scope='a')

rec_a = generator.generator(fake_b, phase_in=phase, scope='b2a')  # reconstructing unfixed-BF from fake-fixed-BF

rec_b = generator.generator(fake_a, phase_in=phase, scope='a2b')  # reconstructing fixed-BF from fake-unfixed-BF

gen_a2b_loss = tf.reduce_mean(tf.losses.mean_squared_error(fake_b_dis, tf.ones_like(fake_b_dis)))
gen_b2a_loss = tf.reduce_mean(tf.losses.mean_squared_error(fake_a_dis, tf.ones_like(fake_a_dis)))

# cycle_loss_unfixed = tf.reduce_mean(tf.abs(a_real - rec_a))
# cycle_loss_fixed = tf.reduce_mean(tf.abs(b_real - rec_b))

cycle_loss_unfixed = tf.reduce_mean(tf.losses.mean_squared_error(a_real,rec_a))
cycle_loss_fixed = tf.reduce_mean(tf.losses.mean_squared_error(b_real,rec_b))

# final generator loss
g_loss =  (gen_a2b_loss + gen_b2a_loss) + 10 * (cycle_loss_unfixed + cycle_loss_fixed)   # FOR deepDeconv
# g_loss =  (gen_a2b_loss + gen_b2a_loss) + 10 * (cycle_loss_unfixed + cycle_loss_fixed)   # FOR das_despeckle
# g_loss =  (gen_a2b_loss + gen_b2a_loss) + 20 * (cycle_loss_unfixed + cycle_loss_fixed)   # FOR deconv_despeckle
learning_rate=0.0002
# g_loss =  (gen_a2b_loss + gen_b2a_loss) + 20 * (cycle_loss_unfixed + cycle_loss_fixed)   # FOR deepDeconv
# learning_rate=0.0005
# learning_rate=0.0001
# learning_rate=0.00001

a_fake_sample = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])
b_fake_sample = tf.placeholder(tf.float32, shape=[None, 256, 256, 3])

dis_a_real = discriminator.discriminator(a_real, training=phase, scope='a')
dis_b_real = discriminator.discriminator(b_real, training=phase, scope='b')

dis_a_fake_sample = discriminator.discriminator(a_fake_sample, training=phase, scope='a')
dis_b_fake_sample = discriminator.discriminator(b_fake_sample, training=phase, scope='b')

# # discriminator loss for liver
da_loss_real = tf.reduce_mean(tf.losses.mean_squared_error(dis_a_real, tf.ones_like(dis_a_real)))
da_loss_fake = tf.reduce_mean(tf.losses.mean_squared_error(dis_a_fake_sample, tf.zeros_like(dis_a_fake_sample)))
da_loss = (da_loss_real + da_loss_fake) / 2

db_loss_real = tf.reduce_mean(tf.losses.mean_squared_error(dis_b_real, tf.ones_like(dis_b_real)))
db_loss_fake = tf.reduce_mean(tf.losses.mean_squared_error(dis_b_fake_sample, tf.zeros_like(dis_b_fake_sample)))
db_loss = (db_loss_real + db_loss_fake) / 2

# final discriminator loss
d_loss = da_loss + db_loss

# summaries
gen_a2b_loss_sum = tf.summary.scalar("gen_a2b_loss", gen_a2b_loss)
gen_b2a_loss_sum = tf.summary.scalar("gen_b2a_loss", gen_b2a_loss)


cycle_loss_unfixed_sum = tf.summary.scalar("cycle_loss_unfixed", cycle_loss_unfixed)
cycle_loss_fixed_sum = tf.summary.scalar("cycle_loss_fixed", cycle_loss_fixed)

g_loss_sum = tf.summary.scalar("g_loss", g_loss)
g_sum = tf.summary.merge([gen_b2a_loss_sum, g_loss_sum, gen_a2b_loss_sum,
                          cycle_loss_unfixed_sum, cycle_loss_fixed_sum])

d_loss_sum = tf.summary.scalar("d_loss", d_loss)

db_loss_real_sum = tf.summary.scalar("db_loss_real", db_loss_real)
db_loss_fake_sum = tf.summary.scalar("db_loss_fake", db_loss_fake)
db_loss_sum = tf.summary.scalar("db_loss", db_loss)

da_loss_real_sum = tf.summary.scalar("da_loss_real", da_loss_real)
da_loss_fake_sum = tf.summary.scalar("da_loss_fake", da_loss_fake)
da_loss_sum = tf.summary.scalar("da_loss", da_loss)

d_sum = tf.summary.merge([da_loss_sum, da_loss_real_sum, da_loss_fake_sum,
                          d_loss_sum, db_loss_sum, db_loss_real_sum, db_loss_fake_sum])

# optimizer
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'discriminator' in var.name]
g_vars = [var for var in t_vars if 'generator' in var.name]
for var in t_vars:
    print(var.name)
# d_optim = tf.train.AdamOptimizer().minimize(d_loss, var_list=d_vars)
# g_optim = tf.train.AdamOptimizer().minimize(g_loss, var_list=g_vars)
d_optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)

# initialize all valuables
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
init = tf.global_variables_initializer()

resume_training = True

Instructions for updating:
Colocations handled automatically by placer.


/content/drive/MyDrive/COLAB_DRIVE/CycleGAN_Denoising_Code/architecture/generator.py:17: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  en = tf.layers.batch_normalization(en, momentum=0.9, training=training, name='bn')
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/normalization.py:463: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs, training=training)
/content/drive/MyDrive/COLAB_DRIVE/CycleGAN_Denoising_Code/architecture/discriminator.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particu

a2b_generator/en1_1/DW:0
a2b_generator/en1_1/bias:0
a2b_generator/en1_1/bn/gamma:0
a2b_generator/en1_1/bn/beta:0
a2b_generator/en1_2/DW:0
a2b_generator/en1_2/bias:0
a2b_generator/en1_2/bn/gamma:0
a2b_generator/en1_2/bn/beta:0
a2b_generator/en2_2/DW:0
a2b_generator/en2_2/bias:0
a2b_generator/en2_2/bn/gamma:0
a2b_generator/en2_2/bn/beta:0
a2b_generator/en2_3/DW:0
a2b_generator/en2_3/bias:0
a2b_generator/en2_3/bn/gamma:0
a2b_generator/en2_3/bn/beta:0
a2b_generator/en3_2/DW:0
a2b_generator/en3_2/bias:0
a2b_generator/en3_2/bn/gamma:0
a2b_generator/en3_2/bn/beta:0
a2b_generator/en3_3/DW:0
a2b_generator/en3_3/bias:0
a2b_generator/en3_3/bn/gamma:0
a2b_generator/en3_3/bn/beta:0
a2b_generator/en4_2/DW:0
a2b_generator/en4_2/bias:0
a2b_generator/en4_2/bn/gamma:0
a2b_generator/en4_2/bn/beta:0
a2b_generator/en4_3/DW:0
a2b_generator/en4_3/bias:0
a2b_generator/en4_3/bn/gamma:0
a2b_generator/en4_3/bn/beta:0
a2b_generator/en5_2/DW:0
a2b_generator/en5_2/bias:0
a2b_generator/en5_2/bn/gamma:0
a2b_generator

In [ ]:
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    sess.run(init)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=100)
    sess.graph.finalize()
    dir_save = "./training_weights/"
    utils.mkdir(dir_save)

    print(' [*] Loading checkpoint...')
    checkpoint = tf.train.latest_checkpoint(dir_save)
    indicie = int(0)
    if resume_training is False:
        print(' [*] Starting training')
    elif checkpoint:
        print(" [*] Loading succeeds! Copy variables from % s", checkpoint)
        c = checkpoint.find('progress-')
        indicie = int(checkpoint[c + 9:]) + 1
        e_count = int(indicie / num_train)
        print(indicie)
        saver.restore(sess, checkpoint)
    else:
        print(' [*] No checkpoint. Starting training')
        e_count = 0
    print("starting indicie : ", indicie)
    dir_ = "./summaries/"
    utils.mkdir(dir_)
    summary_writer = tf.summary.FileWriter(dir_, sess.graph)
    counter = 0

    for epoch in range(n_epochs):
        ec = e_count + epoch
        for run in range(0, num_train, batch_size):
            a_real_ipt, b_real_ipt = utils.train_next_batch(batch_size, num_train, train_img, train_labels)
            # run g_a2b
            fake_b_, rec_b_ = sess.run([fake_b,rec_b], feed_dict={a_real: a_real_ipt,b_real: b_real_ipt, phase: True})
            # run g_b2a
            fake_a_, rec_a_ = sess.run([fake_a, rec_a], feed_dict={a_real: a_real_ipt,b_real: b_real_ipt, phase: True})

            counter += 1
            _, g_summary_str = sess.run([g_optim, g_sum],
                                        feed_dict={a_real: a_real_ipt, b_real: b_real_ipt, phase: True})

            summary_writer.add_summary(g_summary_str, global_step=indicie + counter)

            _, d_summary_str = sess.run([d_optim, d_sum],
                                        feed_dict={a_real: a_real_ipt, b_real: b_real_ipt,
                                                   a_fake_sample: fake_a_, b_fake_sample: fake_b_, phase: True})

            summary_writer.add_summary(d_summary_str, global_step=indicie + counter)

            print("Epoch: (%2d) (%2d) [%2d / %2d]" % (ec, indicie + counter, run, num_train))
            # sample
            if (indicie + counter) % 5 == 0:#508 == 0:
                save_dir = './progress/'
                utils.mkdir(save_dir)
                data_in = 255 * a_real_ipt
                img_in = data_in.astype(np.uint8)
                data_tar = 255 * b_real_ipt
                img_tar = data_tar.astype(np.uint8)
                data_out = 255 * fake_b_
                img_out = data_out.astype(np.uint8)
                if(batch_size>1):
                  utils.im_write(img_in[0], '%s/input.bmp' % save_dir)
                  utils.im_write(img_tar[0], '%s/target.bmp' % save_dir)
                  utils.im_write(img_out[0], '%s/output.bmp' % save_dir)
                else:
                  utils.im_write(img_in, '%s/input.bmp' % save_dir)
                  utils.im_write(img_tar, '%s/target.bmp' % save_dir)
                  utils.im_write(img_out, '%s/output.bmp' % save_dir)

                # save
            if (indicie + counter) % 400 == 0:
                save_path = saver.save(sess, dir_save + "progress", global_step=indicie + counter)
                print('Model saved in file: % s' % save_path)
                print("Here you can include whatever code you like .....")


 [*] Loading checkpoint...
 [*] No checkpoint. Starting training
starting indicie :  0
